<img src="https://fsdl.me/logo-720-dark-horizontal">

# Lab 02b: Training a CNN on Synthetic Handwriting Data

### What You Will Learn

- Fundamental principles for building neural networks with convolutional components
- How to use Lightning's training framework via a CLI

## Setup

If you're running this notebook on Google Colab,
the cell below will run full environment setup.

It should take about three minutes to run.

In [1]:
lab_idx = 2

if "bootstrap" not in locals() or bootstrap.run:
    # path management for Python
    pythonpath, = !echo $PYTHONPATH
    if "." not in pythonpath.split(":"):
        pythonpath = ".:" + pythonpath
        %env PYTHONPATH={pythonpath}
        !echo $PYTHONPATH

    # get both Colab and local notebooks into the same state
    !wget --quiet https://fsdl.me/gist-bootstrap -O bootstrap.py
    import bootstrap

    # change into the lab directory
    bootstrap.change_to_lab_dir(lab_idx=lab_idx)

    # allow "hot-reloading" of modules
    %load_ext autoreload
    %autoreload 2
    # needed for inline plots in some contexts
    %matplotlib inline

    bootstrap.run = False  # change to True re-run setup
    
!pwd
%ls

/home/alotaima/Projects/github/fsdl-text-recognizer-2022-labs/lab02
lightning_logs/  notebooks/  text_recognizer/  training/


# Why convolutions?

The most basic neural networks,
multi-layer perceptrons,
are built by alternating
parameterized linear transformations
with non-linear transformations.

This combination is capable of expressing
[functions of arbitrary complexity](http://neuralnetworksanddeeplearning.com/chap4.html),
so long as those functions
take in fixed-size arrays and return fixed-size arrays.

```python
def any_function_you_can_imagine(x: torch.Tensor["A"]) -> torch.Tensor["B"]:
    return some_mlp_that_might_be_impractically_huge(x)
```

But not all functions have that type signature.

For example, we might want to identify the content of images
that have different sizes.
Without gross hacks,
an MLP won't be able to solve this problem,
even though it seems simple enough.

In [2]:
import random

import IPython.display as display

randsize = 10 ** (random.random() * 2 + 1)

Url = "https://fsdl-public-assets.s3.us-west-2.amazonaws.com/emnist/U.png"

# run multiple times to display the same image at different sizes
#  the content of the image remains unambiguous
display.Image(url=Url, width=randsize, height=randsize)

Even worse, MLPs are too general to be efficient.

Each layer applies an unstructured matrix to its inputs.
But most of the data we might want to apply them to is highly structured,
and taking advantage of that structure can make our models more efficient.

It may seem appealing to use an unstructured model:
it can in principle learn any function.
But
[most functions are monstrous outrages against common sense](https://en.wikipedia.org/wiki/Weierstrass_function#Density_of_nowhere-differentiable_functions).
It is useful to encode some of our assumptions
about the kinds of functions we might want to learn
from our data into our model's architecture.

## Convolutions are the local, translation-equivariant linear transforms.

One of the most common types of structure in data is "locality" --
the most relevant information for understanding or predicting a pixel
is a small number of pixels around it.

Locality is a fundamental feature of the physical world,
so it shows up in data drawn from physical observations,
like photographs and audio recordings.

Locality means most meaningful linear transformations of our input
only have large weights in a small number of entries that are close to one another,
rather than having equally large weights in all entries.

In [3]:
import torch


generic_linear_transform = torch.randn(8, 1)
print("generic:", generic_linear_transform, sep="\n")

local_linear_transform = torch.tensor([
    [0, 0, 0] + [random.random(), random.random(), random.random()] + [0, 0]]).T
print("local:", local_linear_transform, sep="\n")

generic:
tensor([[ 0.4463],
        [-0.5018],
        [-0.4915],
        [-0.7148],
        [-1.2560],
        [ 0.8333],
        [ 1.5323],
        [-1.1543]])
local:
tensor([[0.0000],
        [0.0000],
        [0.0000],
        [0.9529],
        [0.1544],
        [0.2086],
        [0.0000],
        [0.0000]])


Another type of structure commonly observed is "translation equivariance" --
the top-left pixel position is not, in itself, meaningfully different
from the bottom-right position
or a position in the middle of the image.
Relative relationships matter more than absolute relationships.

Translation equivariance arises in images because there is generally no privileged
vantage point for taking the image.
We could just as easily have taken the image while standing a few feet to the left or right,
and all of its contents would shift along with our change in perspective.

Translation equivariance means that a linear transformation that is meaningful at one position
in our input is likely to be meaningful at all other points.
We can learn something about a linear transformation from a datapoint where it is useful
in the bottom-left and then apply it to another datapoint where it's useful in the top-right.

In [4]:
generic_linear_transform = torch.arange(8)[:, None]
print("generic:", generic_linear_transform, sep="\n")

equivariant_linear_transform = torch.stack([torch.roll(generic_linear_transform[:, 0], ii) for ii in range(8)], dim=1)
print("translation invariant:", equivariant_linear_transform, sep="\n")

generic:
tensor([[0],
        [1],
        [2],
        [3],
        [4],
        [5],
        [6],
        [7]])
translation invariant:
tensor([[0, 7, 6, 5, 4, 3, 2, 1],
        [1, 0, 7, 6, 5, 4, 3, 2],
        [2, 1, 0, 7, 6, 5, 4, 3],
        [3, 2, 1, 0, 7, 6, 5, 4],
        [4, 3, 2, 1, 0, 7, 6, 5],
        [5, 4, 3, 2, 1, 0, 7, 6],
        [6, 5, 4, 3, 2, 1, 0, 7],
        [7, 6, 5, 4, 3, 2, 1, 0]])


A linear transformation that is translation equivariant
[is called a _convolution_](https://en.wikipedia.org/wiki/Convolution#Translational_equivariance).

If the weights of that linear transformation are mostly zero
except for a few that are close to one another,
that convolution is said to have a _kernel_.

In [5]:
# the equivalent of torch.nn.Linear, but for a 1-dimensional convolution
conv_layer = torch.nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3)

conv_layer.weight  # aka kernel

Parameter containing:
tensor([[[ 0.3085, -0.4343,  0.1192]]], requires_grad=True)

Instead of using normal matrix multiplication to apply the kernel to the input,
we repeatedly apply that kernel over and over again,
"sliding" it over the input to produce an output.

Every convolution kernel has an equivalent matrix form,
which can be matrix multiplied with the input to create the output:

In [6]:
conv_kernel_as_vector = torch.hstack([conv_layer.weight[0][0], torch.zeros(5)])
conv_layer_as_matrix = torch.stack([torch.roll(conv_kernel_as_vector, ii) for ii in range(8)], dim=0)
print("convolution matrix:", conv_layer_as_matrix, sep="\n")

convolution matrix:
tensor([[ 0.3085, -0.4343,  0.1192,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.3085, -0.4343,  0.1192,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.3085, -0.4343,  0.1192,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.3085, -0.4343,  0.1192,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.3085, -0.4343,  0.1192,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.3085, -0.4343,  0.1192],
        [ 0.1192,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.3085, -0.4343],
        [-0.4343,  0.1192,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.3085]],
       grad_fn=<StackBackward0>)


> <small> Under the hood, the actual operation that implements the application of a convolutional kernel
need not look like either of these
(common approaches include
[Winograd-type algorithms](https://arxiv.org/abs/1509.09308)
and [Fast Fourier Transform-based algorithms](https://arxiv.org/abs/1312.5851)).</small>

Though they may seem somewhat arbitrary and technical,
convolutions are actually a deep and fundamental piece of mathematics and computer science.
Fundamental as in
[closely related to the multiplication algorithm we learn as children](https://charlesfrye.github.io/math/2019/02/20/multiplication-convoluted-part-one.html)
and deep as in
[closely related to the Fourier transform](https://math.stackexchange.com/questions/918345/fourier-transform-as-diagonalization-of-convolution).
Generalized convolutions can show up 
wherever there is some kind of "sum" over some kind of "paths",
as is common in dynamic programming.

See Chris Olah's blog series
([1](https://colah.github.io/posts/2014-07-Conv-Nets-Modular/),
[2](https://colah.github.io/posts/2014-07-Understanding-Convolutions/),
[3](https://colah.github.io/posts/2014-12-Groups-Convolution/))
for a friendly introduction.

## We apply two-dimensional convolutions to images.

In building our text recognizer,
we're working with images.
Images have two dimensions of translation equivariance:
left/right and up/down.
So we use two-dimensional convolutions,
instantiated in `torch.nn` as `nn.Conv2d` layers.
Note that convolutional neural networks for images
are so popular that when the term "convolution"
is used without qualifier in a neural network context,
it can be taken to mean two-dimensional convolutions.

Where `Linear` layers took in batches of vectors of a fixed size
and returned batches of vectors of a fixed size,
`Conv2d` layers take in batches of two-dimensional _stacked feature maps_
and return batches of two-dimensional stacked feature maps.

A pseudocode type signature based on
[`torchtyping`](https://github.com/patrick-kidger/torchtyping)
might look like:

```python
StackedFeatureMapIn = torch.Tensor["batch", "in_channels", "in_height", "in_width"]
StackedFeatureMapOut = torch.Tensor["batch", "out_channels", "out_height", "out_width"]
def same_convolution_2d(x: StackedFeatureMapIn) -> StackedFeatureMapOut:
```

Here, "map" is meant to evoke space:
our feature maps tell us where
features are spatially located.

An RGB image is a stacked feature map.
It is composed of three feature maps.
The first tells us where the "red" feature is present,
the second "green", the third "blue":

In [7]:
display.Image(
    url="https://upload.wikimedia.org/wikipedia/commons/5/56/RGB_channels_separation.png?20110219015028")

When we apply a convolutional layer to a stacked feature map with some number of channels,
we get back a stacked feature map with some number of channels.

This output is also a stack of feature maps,
and so it is a perfectly acceptable
input to another convolutional layer.
That means we can compose convolutional layers together,
just as we composed generic linear layers together.
We again weave non-linear functions in between our linear convolutions,
creating a _convolutional neural network_, or CNN.

## Convolutional neural networks build up visual understanding layer by layer.

What is the equivalent of the labels, red/green/blue,
for the channels in these feature maps?
What does a high activation in some position in channel 32
of the fifteenth layer of my network tell me?

There is no guaranteed way to automatically determine the answer,
nor is there a guarantee that the result is human-interpetable.
OpenAI's Clarity team spent several years "reverse engineering"
state-of-the-art convolutiuonal neural networks trained on photographs
and found that many of these channels are 
[directly interpretable](https://distill.pub/2018/building-blocks/).

For example, they found that if they pass an image through
[GoogLeNet](https://doi.org/10.1109/cvpr.2015.7298594),
aka InceptionV1,
the winner of the
[2014 ImageNet Very Large Scale Visual Recognition Challenge](https://www.image-net.org/challenges/LSVRC/2014/),

In [8]:
# a sample image
display.Image(url="https://distill.pub/2018/building-blocks/examples/input_images/dog_cat.jpeg")

the features become increasingly complex,
with channels in early layers (left)
acting as maps for simple things like "high frequency power" or "45 degree black-white edge"
and channels in later layers (to right)
acting as feature maps for increasingly abstract concepts,
like "circle" and eventually "floppy round ear" or "pointy ear":

In [9]:
# from https://distill.pub/2018/building-blocks/
display.Image(url="https://fsdl-public-assets.s3.us-west-2.amazonaws.com/distill-feature-attrib.png", width=1024)

> <small> The small square images depict a heuristic estimate
of what the entire collection of feature maps
at a given layer represent (layer IDs at bottom).
They are arranged in a spatial grid and their sizes represent
the total magnitude of the layer's activations at that position.
For details and interactivity, see
[the original Distill article](https://distill.pub/2018/building-blocks/).</small>

In the
[Circuits Thread](https://distill.pub/2020/circuits/)
blogpost series,
the Open AI Clarity team
combines careful examination of weights
with direct experimentation
to build an understanding of how these higher-level features
are constructed in GoogLeNet.

For example,
they are able to provide reasonable interpretations for
[almost every channel in the first five layers](https://distill.pub/2020/circuits/early-vision/).

The cell below will pull down their "weight explorer" 
and embed it in this notebook.
By default, it starts on
[the 52nd channel in the `conv2d1` layer](https://storage.googleapis.com/distill-circuits/inceptionv1-weight-explorer/conv2d1_52.html),
which constructs a large, phase-invariant
[Gabor filter](https://en.wikipedia.org/wiki/Gabor_filter)
from smaller, phase-sensitive filters.
It is in turn used to construct
[curve](https://storage.googleapis.com/distill-circuits/inceptionv1-weight-explorer/conv2d2_180.html)
and
[texture](https://storage.googleapis.com/distill-circuits/inceptionv1-weight-explorer/conv2d2_114.html)
detectors --
click on any image to navigate to the weight explorer page
for that channel
or change the `layer` and `idx`
arguments.
For additional context,
check out the
[Early Vision in InceptionV1 blogpost](https://distill.pub/2020/circuits/early-vision/).

Click the "View this neuron in the OpenAI Microscope" link
for an even richer interactive view,
including activations on sample images
([example](https://microscope.openai.com/models/inceptionv1/conv2d1_0/52)).

The
[Circuits Thread](https://distill.pub/2020/circuits/)
which this explorer accompanies
is chock-full of empirical observations, theoretical speculation, and nuggets of wisdom
that are invaluable for developing intuition about both
convolutional networks in particular and visual perception in general.

In [10]:
layers = ["conv2d0", "conv2d1", "conv2d2", "mixed3a", "mixed3b"]
layer = layers[1]
idx = 52

weight_explorer = display.IFrame(
    src=f"https://storage.googleapis.com/distill-circuits/inceptionv1-weight-explorer/{layer}_{idx}.html", width=1024, height=720)
weight_explorer.iframe = 'style="background: #FFF";\n><'.join(weight_explorer.iframe.split("><"))  # inject background color
weight_explorer

# Applying convolutions to handwritten characters: `CNN`s on `EMNIST`

If we load up the `CNN` class from `text_recognizer.models`,
we'll see that a `data_config` is required to instantiate the model:

In [11]:
import text_recognizer.models


text_recognizer.models.CNN??

Init signature:
text_recognizer.models.CNN(
    data_config: Dict[str, Any],
    args: argparse.Namespace = None,
) -> None
Source:        
class CNN(nn.Module):
    """Simple CNN for recognizing characters in a square image."""

    def __init__(self, data_config: Dict[str, Any], args: argparse.Namespace = None) -> None:
        super().__init__()
        self.args = vars(args) if args is not None else {}
        self.data_config = data_config

        input_channels, input_height, input_width = self.data_config["input_dims"]
        assert (
            input_height == input_width
        ), f"input height and width should be equal, but was {input_height}, {input_width}"
        self.input_height, self.input_width = input_height, input_width

        num_classes = len(self.data_config["mapping"])

        conv_dim = self.args.get("conv_dim", CONV_DIM)
        fc_dim = self.args.get("fc_dim", FC_DIM)
        fc_dropout = self.args.get("fc_dropout", FC_DROPOUT)

        self.conv1 = Co

So before we can make our convolutional network and train it,
we'll need to get a hold of some data.
This isn't a general constraint by the way --
it's an implementation detail of the `text_recognizer` library.
But datasets and models are generally coupled,
so it's common for them to share configuration information.

## The `EMNIST` Handwritten Character Dataset

We could just use `MNIST` here,
as we did in
[the first lab](https://fsdl.me/lab01-colab).

But we're aiming to eventually build a handwritten text recognition system,
which means we need to handle letters and punctuation,
not just numbers.

So we instead use _EMNIST_,
or [Extended MNIST](https://paperswithcode.com/paper/emnist-an-extension-of-mnist-to-handwritten),
which includes letters and punctuation.

In [12]:
import text_recognizer.data


emnist = text_recognizer.data.EMNIST()  # configure
print(emnist.__doc__)

EMNIST dataset of handwritten characters and digits.

    "The EMNIST dataset is a set of handwritten character digits derived from the NIST Special Database 19
    and converted to a 28x28 pixel image format and dataset structure that directly matches the MNIST dataset."
    From https://www.nist.gov/itl/iad/image-group/emnist-dataset

    The data split we will use is
    EMNIST ByClass: 814,255 characters. 62 unbalanced classes.
    


We've built a PyTorch Lightning `DataModule`
to encapsulate all the code needed to get this dataset ready to go:
downloading to disk,
[reformatting to make loading faster](https://www.h5py.org/),
and splitting into training, validation, and test.

In [13]:
emnist.prepare_data()  # download, save to disk
emnist.setup()  # create torch.utils.data.Datasets, do train/val split

709MB [00:29, 25.4MB/s]                              


Computing SHA-256...
Unzipping EMNIST...
Loading training data from .mat file
Balancing classes to reduce amount of data
Saving to HDF5 in a compressed format...
Saving essential dataset parameters to text_recognizer/data...
Cleaning up...


A brief aside: you might be wondering where this data goes.
Datasets are saved to disk inside the repo folder,
but not tracked in version control.
`git` works well for versioning source code
and other text files, but it's a poor fit for large binary data.
We only track and version metadata.

In [15]:
!echo {emnist.data_dirname()}
!ls {emnist.data_dirname()}
!ls {emnist.data_dirname() / "raw" / "emnist"}

/home/alotaima/Projects/github/fsdl-text-recognizer-2022-labs/data
downloaded  processed  raw
metadata.toml  readme.md


This class comes with a pretty printing method
for quick examination of some of that metadata and basic descriptive statistics.

In [16]:
emnist

EMNIST Dataset
Num classes: 83
Mapping: ['<B>', '<S>', '<E>', '<P>', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '!', '"', '#', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '?']
Dims: (1, 28, 28)
Train/val/test sizes: 260212, 65054, 53988
Batch x stats: (torch.Size([128, 1, 28, 28]), torch.float32, tensor(0.), tensor(0.1797), tensor(0.3359), tensor(1.))
Batch y stats: (torch.Size([128]), torch.int64, tensor(4), tensor(65))


> <small> You can add pretty printing to your own Python classes by writing
`__str__` or `__repr__` methods for them.
The former is generally expected to be human-readable,
while the latter is generally expected to be machine-readable;
we've broken with that custom here and used `__repr__`. </small>

Because we've run `.prepare_data` and `.setup`,
we can expect that this `DataModule` is ready to provide a `DataLoader`
if we invoke the right method --
sticking to the PyTorch Lightning API brings these kinds of convenient guarantees
even when we're not using the `Trainer` class itself,
[as described in Lab 2a](https://fsdl.me/lab02a-colab).

In [17]:
xs, ys = next(iter(emnist.train_dataloader()))

Run the cell below to inspect random elements of this batch.

In [18]:
import wandb

idx = random.randint(0, len(xs) - 1)

print(emnist.mapping[ys[idx]])
wandb.Image(xs[idx]).image

a


## Putting convolutions in a `torch.nn.Module`

Because we have the data,
we now have a `data_config`
and can instantiate the model.

In [19]:
data_config = emnist.config()

cnn = text_recognizer.models.CNN(data_config)
cnn  # reveals the nn.Modules attached to our nn.Module

CNN(
  (conv1): ConvBlock(
    (conv): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu): ReLU()
  )
  (conv2): ConvBlock(
    (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu): ReLU()
  )
  (dropout): Dropout(p=0.25, inplace=False)
  (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=12544, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=83, bias=True)
)

We can run this network on our inputs,
but we don't expect it to produce correct outputs without training.

In [20]:
idx = random.randint(0, len(xs) - 1)
outs = cnn(xs[idx:idx+1])

print("output:", emnist.mapping[torch.argmax(outs)])
wandb.Image(xs[idx]).image

output: m


We can inspect the `.forward` method to see how these `nn.Module`s are used.

> <small> Note: we encourage you to read through the code --
either inside the notebooks, as below,
in your favorite text editor locally, or
[on GitHub](https://github.com/full-stack-deep-learning/fsdl-text-recognizer-2022-labs).
There's lots of useful bits of Python that we don't have time to cover explicitly in the labs.</small>

In [ ]:
cnn.forward??

We apply convolutions followed by non-linearities,
with intermittent "pooling" layers that apply downsampling --
similar to the 1989
[LeNet](https://doi.org/10.1162%2Fneco.1989.1.4.541)
architecture or the 2012
[AlexNet](https://doi.org/10.1145%2F3065386)
architecture.

The final classification is performed by an MLP.

In order to get vectors to pass into that MLP,
we first apply `torch.flatten`.

In [ ]:
torch.flatten(torch.Tensor([[1, 2], [3, 4]]))

## Design considerations for CNNs

Since the release of AlexNet,
there has been a feverish decade of engineering and innovation in CNNs --
[dilated convolutions](https://arxiv.org/abs/1511.07122),
[residual connections](https://arxiv.org/abs/1512.03385), and
[batch normalization](https://arxiv.org/abs/1502.03167)
came out in 2015 alone, and
[work continues](https://arxiv.org/abs/2201.03545) --
so we can only scratch the surface in this course and
[the devil is in the details](https://arxiv.org/abs/1405.3531v4).

The progress of DNNs in general and CNNs in particular
has been mostly evolutionary,
with lots of good ideas that didn't work out
and weird hacks that stuck around because they did.
That can make it very hard to design a fresh architecture
from first principles that's anywhere near as effective as existing architectures.
You're better off tweaking and mutating an existing architecture
than trying to design one yourself.

If you're not keeping close tabs on the field,
when your first start looking for an architecture to base your work off of
it's best to go to trusted aggregators, like
[Torch IMage Models](https://github.com/rwightman/pytorch-image-models),
or `timm`, on GitHub, or
[Papers With Code](https://paperswithcode.com),
specifically the section for
[computer vision](https://paperswithcode.com/methods/area/computer-vision).
You can also take a more bottom-up approach by checking
the leaderboards of the latest
[Kaggle competitions on computer vision](https://www.kaggle.com/competitions?searchQuery=computer+vision).

We'll briefly touch here on some of the main design considerations
with classic CNN architectures.

### Shapes and padding

In the `.forward` pass of the `CNN`,
we've included comments that indicate the expected shapes
of tensors after each line that changes the shape.

Tracking and correctly handling shapes is one of the bugbears
of CNNs, especially architectures,
like LeNet/AlexNet, that include MLP components
that can only operate on fixed-shape tensors.

[Shape arithmetic gets pretty hairy pretty fast](https://arxiv.org/abs/1603.07285)
if you're supporting the wide variety of convolutions.

The easiest way to avoid shape bugs is to keep things simple: 
choose your convolution parameters,
like `padding` and `stride`,
to keep the shape the same before and after
the convolution.

That's what we do, by choosing `padding=1`
for `kernel_size=3` and `stride=1`.
With unit strides and odd-numbered kernel size,
the padding that keeps
the input the same size is `kernel_size // 2`.

As shapes change, so does the amount of GPU memory taken up by the tensors.
Keeping sizes fixed within a block removes one axis of variation
in the demands on an important resource.

After applying our pooling layer,
we can just increase the number of kernels by the right factor
to keep total tensor size,
and thus memory footprint, constant.

### Parameters, computation, and bottlenecks

If we review the `num`ber of `el`ements in each of the layers,
we see that one layer has far more entries than all the others:

In [21]:
[p.numel() for p in cnn.parameters()]  # conv weight + bias, conv weight + bias, fc weight + bias, fc weight + bias

[576, 64, 36864, 64, 1605632, 128, 10624, 83]

The biggest layer is typically
the one in between the convolutional component
and the MLP component:

In [22]:
biggest_layer = [p for p in cnn.parameters() if p.numel() == max(p.numel() for p in cnn.parameters())][0]
biggest_layer.shape, cnn.fc_input_dim

(torch.Size([128, 12544]), 12544)

This layer dominates the cost of storing the network on disk.
That makes it a common target for
regularization techniques like DropOut
(as in our architecture)
and performance optimizations like
[pruning](https://pytorch.org/tutorials/intermediate/pruning_tutorial.html).

Heuristically, we often associated more parameters with more computation.
But just because that layer has the most parameters
does not mean that most of the compute time is spent in that layer.

Convolutions reuse the same parameters over and over,
so the total number of FLOPs done by the layer can be higher
than that done by layers with more parameters --
much higher.

In [23]:
# for the Linear layers, number of multiplications per input == nparams
cnn.fc1.weight.numel()

1605632

In [24]:
# for the Conv2D layers, it's more complicated

def approx_conv_multiplications(kernel_shape, input_size=(64, 28, 28)):  # this is a rough and dirty approximation
    num_kernel_elements = 1
    for dimension in kernel_shape[-3:]:
        num_kernel_elements *= dimension
    num_input_channels, num_kernels = input_size[0], kernel_shape[0]
    num_spatial_applications = ((input_size[1] - kernel_shape[-2] + 1) * (input_size[2] - kernel_shape[-1] + 1))
    mutliplications_per_kernel = num_spatial_applications * num_kernel_elements * num_input_channels
    return mutliplications_per_kernel * num_kernels

In [25]:
approx_conv_multiplications(cnn.conv2.conv.weight.shape)

1594884096

In [26]:
# ratio of multiplications in the convolution to multiplications in the fully-connected layer is huge!
approx_conv_multiplications(cnn.conv2.conv.weight.shape) // cnn.fc1.weight.numel()

993

Depending on your compute hardware and the problem characteristics,
either the MLP component or the convolutional component
could become the critical bottleneck.

When you're memory constrained, like when transferring a model "over the wire" to a browser,
the MLP component is likely to be the bottleneck,
whereas when you are compute-constrained, like when running a model on a low-power edge device
or in an application with strict low-latency requirements,
the convolutional component is likely to be the bottleneck.


## Training a `CNN` on `EMNIST` with the Lightning `Trainer` and `run_experiment`

We have a model and we have data,
so we could just go ahead and start training in raw PyTorch,
[as we did in Lab 01](https://fsdl.me/lab01-colab).

But as we saw in that lab,
there are good reasons to use a framework
to organize training and provide fixed interfaces and abstractions.
So we're going to use PyTorch Lightning, which is
[covered in detail in Lab 02a](https://fsdl.me/lab02a-colab).

We provide a simple script that implements a command line interface
to training with PyTorch Lightning
using the models and datasets in this repository:
`training/run_experiment.py`.

In [27]:
%run training/run_experiment.py --help

usage: run_experiment.py [--logger [LOGGER]]
                         [--checkpoint_callback [CHECKPOINT_CALLBACK]]
                         [--enable_checkpointing [ENABLE_CHECKPOINTING]]
                         [--default_root_dir DEFAULT_ROOT_DIR]
                         [--gradient_clip_val GRADIENT_CLIP_VAL]
                         [--gradient_clip_algorithm GRADIENT_CLIP_ALGORITHM]
                         [--process_position PROCESS_POSITION]
                         [--num_nodes NUM_NODES]
                         [--num_processes NUM_PROCESSES] [--devices DEVICES]
                         [--gpus GPUS] [--auto_select_gpus [AUTO_SELECT_GPUS]]
                         [--tpu_cores TPU_CORES] [--ipus IPUS]
                         [--log_gpu_memory LOG_GPU_MEMORY]
                         [--progress_bar_refresh_rate PROGRESS_BAR_REFRESH_RATE]
                         [--enable_progress_bar [ENABLE_PROGRESS_BAR]]
                         [--overfit_batches OVERFIT_BATCHES]
   

The `pl.Trainer` arguments come first
and there
[are a lot of them](https://pytorch-lightning.readthedocs.io/en/1.6.3/common/trainer.html),
so if we want to see what's configurable for
our `Model` or our `LitModel`,
we want the last few dozen lines of the help message:

In [28]:
!python training/run_experiment.py --help --model_class CNN --data_class EMNIST  | tail -n 25

                        .. deprecated:: v1.5 Trainer argument
                        ``terminate_on_nan`` was deprecated in v1.5 and will
                        be removed in 1.7. Please use ``detect_anomaly``
                        instead.

Data Args:
  --batch_size BATCH_SIZE
                        Number of examples to operate on per forward step.
                        Default is 128.
  --num_workers NUM_WORKERS
                        Number of additional processes to load data. Default
                        is 48.

Model Args:
  --conv_dim CONV_DIM
  --fc_dim FC_DIM
  --fc_dropout FC_DROPOUT

LitModel Args:
  --optimizer OPTIMIZER
                        optimizer class from torch.optim
  --lr LR
  --one_cycle_max_lr ONE_CYCLE_MAX_LR
  --one_cycle_total_steps ONE_CYCLE_TOTAL_STEPS
  --loss LOSS           loss function from torch.nn.functional


The `run_experiment.py` file is also importable as a module,
so that you can inspect its contents
and play with its component functions in a notebook.

In [29]:
import training.run_experiment


print(training.run_experiment.main.__doc__)


    Run an experiment.

    Sample command:
    ```
    python training/run_experiment.py --max_epochs=3 --gpus='0,' --num_workers=20 --model_class=MLP --data_class=MNIST
    ```

    For basic help documentation, run the command
    ```
    python training/run_experiment.py --help
    ```

    The available command line args differ depending on some of the arguments, including --model_class and --data_class.

    To see which command line args are available and read their documentation, provide values for those arguments
    before invoking --help, like so:
    ```
    python training/run_experiment.py --model_class=MLP --data_class=MNIST --help
    


Let's run training!

Execute the cell below to launch a training job for a CNN on EMNIST with default arguments.

This will take several minutes on commodity hardware,
so feel free to keep reading while it runs.

In [30]:
gpus = int(torch.cuda.is_available())  # use GPUs if they're available

%run training/run_experiment.py --model_class CNN --data_class EMNIST --gpus {gpus}

Missing logger folder: training/logs/lightning_logs
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type      | Params
---------------------------------------------
0 | model          | CNN       | 1.7 M 
1 | model.conv1    | ConvBlock | 640   
2 | model.conv2    | ConvBlock | 36.9 K
3 | model.dropout  | Dropout   | 0     
4 | model.max_pool | MaxPool2d | 0     
5 | model.fc1      | Linear    | 1.6 M 
6 | model.fc2      | Linear    | 10.7 K
7 | train_acc      | Accuracy  | 0     
8 | val_acc        | Accuracy  | 0     
9 | test_acc       | Accuracy  | 0     
---------------------------------------------
1.7 M     Trainable params
0         Non-trainable 

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test/acc            0.7837111949920654
        test/loss           0.5815876722335815
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


Best model saved at: /home/alotaima/Projects/github/fsdl-text-recognizer-2022-labs/lab02/training/logs/lightning_logs/version_0/epoch=0000-validation.loss=0.576.ckpt


The first thing you'll see are a few logger messages from Lightning,
then some info about the hardware you have available and are using.

Then you'll see a summary of your model,
including module names, parameter counts,
and information about model disk size.

`torchmetrics` show up here as well,
since they are also `nn.Module`s.
See [Lab 02a](https://fsdl.me/lab02a-colab)
for details.
We're tracking accuracy on training, validation, and test sets.

You may also see a quick message in the terminal
referencing a "validation sanity check".
PyTorch Lightning runs a few batches of validation data
through the model before the first training epoch.
This helps prevent training runs from crashing
at the end of the first epoch,
which is otherwise the first time validation loops are triggered
and is sometimes hours into training,
by crashing them quickly at the start.

If you want to turn off the check,
use `--num_sanity_val_steps=0`.

Then, you'll see a bar indicating
progress through the training epoch,
alongside metrics like throughput and loss.

When the first (and only) epoch ends,
the model is run on the validation set
and aggregate loss and accuracy are reported to the console.

At the end of training,
we call `Trainer.test`
to check performance on the test set.

We typically see test accuracy around 75-80%

During training, PyTorch Lightning saves _checkpoints_
(file extension `.ckpt`)
that can be used to restart training.

The final line output by `run_experiment`
indicates where the model with the best performance
on the validation set has been saved.

The checkpointing behavior is configured using a
[`ModelCheckpoint` callback](https://pytorch-lightning.readthedocs.io/en/1.6.3/api/pytorch_lightning.callbacks.ModelCheckpoint.html).
The `run_experiment` script picks sensible defaults.

These checkpoints contain the model weights.
We can use them to los the model in the notebook and play around with it.

In [31]:
# we use a sequence of bash commands to get the latest checkpoint's filename
#  by hand, you can just copy and paste it

list_all_log_files = "find training/logs/lightning_logs"  # find avoids issues with \n in filenames
filter_to_ckpts = "grep \.ckpt$"  # regex match on end of line
sort_version_descending = "sort -Vr"  # uses "version" sorting (-V) and reverses (-r)
take_first = "head -n 1"  # the first n elements, n=1

latest_ckpt, = ! {list_all_log_files} | {filter_to_ckpts} | {sort_version_descending} | {take_first}
latest_ckpt

'training/logs/lightning_logs/version_0/epoch=0000-validation.loss=0.576.ckpt'

To rebuild the model,
we need to consider some implementation details of the `run_experiment` script.

We use the parsed command line arguments, the `args`, to build the data and model,
then use all three to build the `LightningModule`.

Any `LightningModule` can be reinstantiated from a checkpoint
using the `load_from_checkpoint` method,
but we'll need to recreate and pass the `args`
in order to reload the model.
(We'll see how this can be automated later).

In [32]:
import training.util
from argparse import Namespace


# if you change around model/data args in the command above, add them here
#  tip: define the arguments as variables, like we've done for gpus
#       and then add those variables to this dict so you don't need to
#       remember to update/copy+paste

args = Namespace(**{
    "model_class": "CNN",
    "data_class": "EMNIST"})


_, cnn = training.util.setup_data_and_model_from_args(args)

reloaded_model = text_recognizer.lit_models.BaseLitModel.load_from_checkpoint(
   latest_ckpt, args=args, model=cnn)

With the model reloads, we can run it on some sample data
and see how it's doing:

In [33]:
idx = random.randint(0, len(xs) - 1)
outs = reloaded_model(xs[idx:idx+1])

print("output:", emnist.mapping[torch.argmax(outs)])
wandb.Image(xs[idx]).image

output: L


I generally see subjectively good performance --
without seeing the labels, I tend to agree with the model's output
more often than the accuracy would suggest,
since some classes, like c and C or o, O, and 0,
are essentially indistinguishable.

We can continue a promising training run from the checkpoint.
Run the cell below to train the model just trained above
for another epoch.
Note that the training loss starts out closs to where it ended
in the previous run.

Paired with cloud storage of checkpoints,
this makes it possible to use
[a cheaper type of cloud instance](https://cloud.google.com/blog/products/ai-machine-learning/reduce-the-costs-of-ml-workflows-with-preemptible-vms-and-gpus)
that can be pre-empted by someone willing to pay more,
which terminates your job.
It's also helpful when using Google Colab for more serious projects --
your training runs are no longer bound by the maximum uptime of a Colab notebook.

In [34]:
latest_ckpt, = ! {list_all_log_files} | {filter_to_ckpts} | {sort_version_descending} | {take_first}


# and we can change the training hyperparameters, like batch size
%run training/run_experiment.py --model_class CNN --data_class EMNIST --gpus {gpus} \
  --batch_size 64 --load_checkpoint {latest_ckpt}

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type      | Params
---------------------------------------------
0 | model          | CNN       | 1.7 M 
1 | model.conv1    | ConvBlock | 640   
2 | model.conv2    | ConvBlock | 36.9 K
3 | model.dropout  | Dropout   | 0     
4 | model.max_pool | MaxPool2d | 0     
5 | model.fc1      | Linear    | 1.6 M 
6 | model.fc2      | Linear    | 10.7 K
7 | train_acc      | Accuracy  | 0     
8 | val_acc        | Accuracy  | 0     
9 | test_acc       | Accuracy  | 0     
---------------------------------------------
1.7 M     Trainable params
0         Non-trainable params
1.7 M     Total params
6.616     Total estima

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test/acc            0.7947136163711548
        test/loss           0.5398882627487183
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


Best model saved at: /home/alotaima/Projects/github/fsdl-text-recognizer-2022-labs/lab02/training/logs/lightning_logs/version_1/epoch=0000-validation.loss=0.534.ckpt


# Creating lines of text from handwritten characters: `EMNISTLines`

We've got a training pipeline for our model and our data,
and we can use that to make the loss go down
and get better at the task.
But the problem we're solving not obviously useful:
the model is just learning hwo to handle
centered, high-contrast, isolated characters.

To make this work in a text recognition application,
we would need a component to first pull out characters like that from images.
That task is probably harder than the one we're currently learning.
Plus, splitting into two separate components is against the ethos of deep learning,
which operates "end-to-end".

Let's kick the realism up one notch by building lines of text out of our characters:
_synthesizing_ data for our model.

Synthetic data is generally useful for augmenting limited real data.
By construction we know the labels, since we created the data.
Often, we can track covariates,
like lighting features or subclass membership,
that aren't always available in our labels.

To build fake handwriting,
we'll combine two things:
real handwritten letters and real text.

We generate our fake text by drawing from the
[Brown corpus](https://en.wikipedia.org/wiki/Brown_Corpus)
provided by the [`n`atural `l`anguage `t`ool`k`it](https://www.nltk.org/) library.

First, we download that corpus.

In [35]:
from text_recognizer.data.sentence_generator import SentenceGenerator

sentence_generator = SentenceGenerator()

SentenceGenerator.__doc__

[nltk_data] Downloading package brown to
[nltk_data]     /home/alotaima/Projects/github/fsdl-text-
[nltk_data]     recognizer-2022-labs/data/downloaded/nltk...
[nltk_data]   Unzipping corpora/brown.zip.


'Generate text sentences using the Brown corpus.'

We can generate short snippets of text from the corpus with the `SentenceGenerator`:

In [36]:
print(*[sentence_generator.generate(max_length=16) for _ in range(4)], sep="\n")

phase
Within
either
the


We use another `DataModule` to pick out the needed handwritten characters from `EMNIST`
and glue them together into images containing the generated text.

In [37]:
emnist_lines = text_recognizer.data.EMNISTLines()  # configure
emnist_lines.__doc__

'EMNIST Lines dataset: synthetic handwriting lines dataset made from EMNIST characters.'

This can take several minutes when first run,
but afterwards data is persisted to disk.

In [38]:
emnist_lines.prepare_data()  # download, save to disk
emnist_lines.setup()  # create torch.utils.data.Datasets, do train/val split
emnist_lines

EMNISTLinesDataset generating data for train...
EMNISTLinesDataset generating data for val...
EMNISTLinesDataset generating data for test...
EMNISTLinesDataset loading data from HDF5...


EMNIST Lines Dataset
Min overlap: 0
Max overlap: 0.33
Num classes: 83
Dims: (1, 28, 896)
Output dims: (32, 1)
Train/val/test sizes: 10000, 2000, 2000
Batch x stats: (torch.Size([128, 1, 28, 896]), torch.float32, 0.0, 0.07364349812269211, 0.23208004236221313, 1.0)
Batch y stats: (torch.Size([128, 32]), torch.int64, 3, 66)

Again, we're using the `LightningDataModule` interface
to organize our data prep,
so we can now fetch a batch and take a look at some data.

In [39]:
line_xs, line_ys = next(iter(emnist_lines.val_dataloader()))
line_xs.shape, line_ys.shape

(torch.Size([128, 1, 28, 896]), torch.Size([128, 32]))

In [40]:
def read_line_labels(labels):
    return [emnist_lines.mapping[label] for label in labels]

idx = random.randint(0, len(line_xs) - 1)

print("-".join(read_line_labels(line_ys[idx])))
wandb.Image(line_xs[idx]).image

b-l-o-s-s-o-m- -w-h-i-l-e- -t-h-e- -s-e-c-o-n-d- -h-a-l-f-<P>-<P>-<P>


The result looks
[kind of like a ransom note](https://tvtropes.org/pmwiki/pmwiki.php/Main/CutAndPasteNote)
and is not yet anywhere near realistic, even for single lines --
letters don't overlap, the exact same handwritten letter is repeated
if the character appears more than once in the snippet --
but it's a start.

# Applying CNNs to handwritten text: `LineCNNSimple`

The `LineCNNSimple` class builds on the `CNN` class and can be applied to this dataset.

In [41]:
line_cnn = text_recognizer.models.LineCNNSimple(emnist_lines.config())
line_cnn

LineCNNSimple(
  (cnn): CNN(
    (conv1): ConvBlock(
      (conv): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (relu): ReLU()
    )
    (conv2): ConvBlock(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (relu): ReLU()
    )
    (dropout): Dropout(p=0.25, inplace=False)
    (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (fc1): Linear(in_features=12544, out_features=128, bias=True)
    (fc2): Linear(in_features=128, out_features=83, bias=True)
  )
)

The `nn.Module`s look much the same,
but the way they are used is different,
which we can see by examining the `.forward` method:

In [42]:
line_cnn.forward??

Signature: line_cnn.forward(x: torch.Tensor) -> torch.Tensor
Source:   
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Apply the LineCNN to an input image and return logits.

        Parameters
        ----------
        x
            (B, C, H, W) input image with H equal to IMAGE_SIZE

        Returns
        -------
        torch.Tensor
            (B, C, S) logits, where S is the length of the sequence and C is the number of classes
            S can be computed from W and CHAR_WIDTH
            C is self.num_classes
        """
        B, _C, H, W = x.shape
        assert H == IMAGE_SIZE  # Make sure we can use our CNN class

        # Compute number of windows
        S = math.floor((W - self.WW) / self.WS + 1)

        # NOTE: type_as properly sets device
        activations = torch.zeros((B, self.num_classes, S)).type_as(x)
        for s in range(S):
            start_w = self.WS * s
            end_w = start_w + self.WW
            window = x[:, :, :, start_

The `CNN`, which operates on square images,
is applied to our wide image repeatedly,
slid over by the `W`indow `S`ize each time.
We effectively convolve the network with the input image.

Like our synthetic data, it is crude
but it's enough to get started.

In [43]:
idx = random.randint(0, len(line_xs) - 1)

outs, = line_cnn(line_xs[idx:idx+1])
preds = torch.argmax(outs, 0)

print("-".join(read_line_labels(preds)))
wandb.Image(line_xs[idx]).image

K-K-N-N-K-N-N-H-H-N-H-N-N-H-N-q-H-H-q-N-L-N-q-q-H-N-N-N-N-q-H-H


> <small> You may notice that this randomly-initialized
network tends to predict some characters far more often than others,
rather than predicting all characters with equal likelihood.
This is a commonly-observed phenomenon in deep networks.
It is connected to issues with
[model calibration](https://arxiv.org/abs/1706.04599)
and Bayesian uses of DNNs
(see e.g. Figure 7 of
[Wenzel et al. 2020](https://arxiv.org/abs/2002.02405).</small>

Let's launch a training run with the default parameters.

This cell should run in just a few minutes on typical hardware.

In [44]:
%run training/run_experiment.py --model_class LineCNNSimple --data_class EMNISTLines \
  --batch_size 32 --gpus {gpus} --max_epochs 2

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


EMNISTLinesDataset loading data from HDF5...


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type          | Params
--------------------------------------------
0 | model     | LineCNNSimple | 1.7 M 
1 | model.cnn | CNN           | 1.7 M 
2 | train_acc | Accuracy      | 0     
3 | val_acc   | Accuracy      | 0     
4 | test_acc  | Accuracy      | 0     
--------------------------------------------
1.7 M     Trainable params
0         Non-trainable params
1.7 M     Total params
6.616     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


EMNISTLinesDataset loading data from HDF5...


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test/acc            0.6750156283378601
        test/loss           1.4330782890319824
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


Best model saved at: /home/alotaima/Projects/github/fsdl-text-recognizer-2022-labs/lab02/training/logs/lightning_logs/version_2/epoch=0001-validation.loss=1.382.ckpt


You should see a test accuracy in the 65-70% range.

That seems pretty good,
especially for a simple model trained in a minute.

Let's reload the model and run it on some examples.

In [45]:
# if you change around model/data args in the command above, add them here
#  tip: define the arguments as variables, like we've done for gpus
#       and then add those variables to this dict so you don't need to
#       remember to update/copy+paste

args = Namespace(**{
    "model_class": "LineCNNSimple",
    "data_class": "EMNISTLines"})


_, line_cnn = training.util.setup_data_and_model_from_args(args)

latest_ckpt, = ! {list_all_log_files} | {filter_to_ckpts} | {sort_version_descending} | {take_first}
print(latest_ckpt)

reloaded_lines_model = text_recognizer.lit_models.BaseLitModel.load_from_checkpoint(
   latest_ckpt, args=args, model=line_cnn)

training/logs/lightning_logs/version_2/epoch=0001-validation.loss=1.382.ckpt


In [46]:
idx = random.randint(0, len(line_xs) - 1)

outs, = reloaded_lines_model(line_xs[idx:idx+1])
preds = torch.argmax(outs, 0)

print("-".join(read_line_labels(preds)))
wandb.Image(line_xs[idx]).image

h-e-r- -t-n- -o-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>


In general,
we see predictions that have very low subjective quality:
it seems like most of the letters are wrong
and the model often prefers to predict the most common letters
in the dataset, like `e`.

Notice, however, that many of the
characters in a given line are padding characters, `<P>`.

A model that always predicts `<P>` can achieve around 50% accuracy:

In [47]:
padding_token = emnist_lines.emnist.inverse_mapping["<P>"]
torch.sum(line_ys == padding_token) / line_ys.numel()

tensor(0.5359)

There are ways to adjust your classification metrics to
[handle this particular issue](https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall).
In general it's good to find a metric
that has baseline performance at 0 and perfect performance at 1,
so that numbers are clearly interpretable.

But it's an important reminder to actually look
at your model's behavior from time to time.
Metrics are single numbers,
so they by necessity throw away a ton of information
about your model's behavior,
some of which is deeply relevant.

# Exercises

### 🌟 Research a `pl.Trainer` argument and try it out.

The Lightning `Trainer` class is highly configurable
and has accumulated a number of features as Lightning has matured.

Check out the documentation for this class
and pick an argument to try out with `training/run_experiment.py`.
Look for edge cases in its behavior,
especially when combined with other arguments.

In [48]:
import pytorch_lightning as pl

pl_version = pl.__version__

print("pl.Trainer guide URL:", f"https://pytorch-lightning.readthedocs.io/en/{pl_version}/common/trainer.html")
print("pl.Trainer reference docs URL:", f"https://pytorch-lightning.readthedocs.io/en/{pl_version}/api/pytorch_lightning.trainer.trainer.Trainer.html")

pl.Trainer??

pl.Trainer guide URL: https://pytorch-lightning.readthedocs.io/en/1.6.3/common/trainer.html
pl.Trainer reference docs URL: https://pytorch-lightning.readthedocs.io/en/1.6.3/api/pytorch_lightning.trainer.trainer.Trainer.html


Init signature:
pl.Trainer(
    logger: Union[pytorch_lightning.loggers.base.LightningLoggerBase, Iterable[pytorch_lightning.loggers.base.LightningLoggerBase], bool] = True,
    checkpoint_callback: Union[bool, NoneType] = None,
    enable_checkpointing: bool = True,
    callbacks: Union[List[pytorch_lightning.callbacks.base.Callback], pytorch_lightning.callbacks.base.Callback, NoneType] = None,
    default_root_dir: Union[str, NoneType] = None,
    gradient_clip_val: Union[int, float, NoneType] = None,
    gradient_clip_algorithm: Union[str, NoneType] = None,
    process_position: int = 0,
    num_nodes: int = 1,
    num_processes: Union[int, NoneType] = None,
    devices: Union[List[int], str, int, NoneType] = None,
    gpus: Union[List[int], str, int, NoneType] = None,
    auto_select_gpus: bool = False,
    tpu_cores: Union[List[int], str, int, NoneType] = None,
    ipus: Union[int, NoneType] = None,
    log_gpu_memory: Union[str, NoneType] = None,
    progress_bar_refresh_rate: Un

In [49]:
%run training/run_experiment.py --help

usage: run_experiment.py [--logger [LOGGER]]
                         [--checkpoint_callback [CHECKPOINT_CALLBACK]]
                         [--enable_checkpointing [ENABLE_CHECKPOINTING]]
                         [--default_root_dir DEFAULT_ROOT_DIR]
                         [--gradient_clip_val GRADIENT_CLIP_VAL]
                         [--gradient_clip_algorithm GRADIENT_CLIP_ALGORITHM]
                         [--process_position PROCESS_POSITION]
                         [--num_nodes NUM_NODES]
                         [--num_processes NUM_PROCESSES] [--devices DEVICES]
                         [--gpus GPUS] [--auto_select_gpus [AUTO_SELECT_GPUS]]
                         [--tpu_cores TPU_CORES] [--ipus IPUS]
                         [--log_gpu_memory LOG_GPU_MEMORY]
                         [--progress_bar_refresh_rate PROGRESS_BAR_REFRESH_RATE]
                         [--enable_progress_bar [ENABLE_PROGRESS_BAR]]
                         [--overfit_batches OVERFIT_BATCHES]
   